In [44]:
import hazm
import parsivar
from string import punctuation
import json
import re
import numpy as np
from pathlib import Path
from collections import Counter
import collections
from hazm import Normalizer, word_tokenize, Stemmer
from collections import Counter
import json
import re
import string
from collections import Counter
from hazm import Normalizer, word_tokenize, Stemmer

In [45]:
def load_docs():
    docs = {}
    contents = []
    urls = []
    with open("IR_data_news_12k.json", 'r') as file:
        docs = json.load(file)
        for key in docs.keys():
            # index of files
            idx = str(key)
            # extract and save url, title and content of each doc
            docs[idx] = {'title': docs[idx]['title'],
                         'content': docs[idx]['content'],
                         'url': docs[idx]['url'],
                         }

            contents.append(docs[idx]['content'])
    return docs, contents, urls

In [46]:
docs, contents, urls = load_docs()
docs['6']

{'title': 'اعلام برنامه نشست خبری گل محمدی/ مجیدی هم باید به محل تمرین پرسپولیس برود!',
 'content': '\nبه گزارش خبرگزاری فارس و به نقل از\xa0 سایت باشگاه پرسپولیس، نشست خبری پیش از مسابقه سرمربیان دو تیم پرسپولیس و استقلال از هفته بیست و سوم لیگ برتر (جام خلیج فارس) با\xa0 مدیریت سازمان لیگ و هماهنگی\xa0باشگاه میزبان (پرسپولیس) در ورزشگاه شهید کاظمی به شرح زیر\xa0 برگزار می\u200cشود: چهارشنبه ۲۵ اسفند ساعت ۱۳ فرهاد مجیدی سرمربی استقلال ساعت ۱۳:۳۰ یحیی گل محمدی سرمربی پرسپولیس \xa0مسابقه دو تیم روز پنجشنبه در ورزشگاه آزادی برگزار می\u200cشود. \xa0به گزارش خبرگزاری فارس، پیش از این باشگاه استقلال اعلام کرده بود قرار است نشست خبری فرهاد مجیدی، سرمربی این تیم از ساعت ۱۵ در سازمان لیگ برگزار شود. انتهای پیام /\n\n\n',
 'url': 'https://www.farsnews.ir/news/14001224000865/اعلام-برنامه-نشست-خبری-گل-محمدی-مجیدی-هم-باید-به-محل-تمرین-پرسپولیس'}

In [47]:
import re
from hazm import Normalizer, word_tokenize, Stemmer
from collections import Counter

ABBREVIATIONS = {
    "دک": "دانشکده",
    "ق.ظ": "قبل از ظهر",
    "خ.ا": "خدمت امام",
}

EMAIL_PATTERN = re.compile(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)")
ID_PATTERN = re.compile(r"(@[\w_]+)")

SPACING_PATTERNS = [
    # Suffixes like ها، هاي، هایی، تر، تري، ترین، ...
    (r"(\S)(ها|هاي|هایی|تر|تري|ترین|گر|گري|ام|ات|اش)(\s|$)", r"\1‌\2 "),
    # mi or nmi => zero-width non-joiner with next token
    (r"\b(می|نمی)\s+(\S)", r"\1‌\2"),
]

DIACRITICS_PATTERN = r"[\u064b\u064c\u064d\u064e\u064f\u0650\u0651\u0652\u0670]+"
EXTRA_PUNCT_PATTERN = r"[!<>.,؛،:\-–_=+(){}\[\]…\"\'?؟«»٪%]+"
normalizer = Normalizer()


def expand_abbreviations(text):
    """Expands abbreviations using the ABBREVIATIONS dictionary."""
    for short_form, expansion in ABBREVIATIONS.items():
        pattern = fr"\b{re.escape(short_form)}\b"
        text = re.sub(pattern, expansion, text)
    return text


def preserve_emails_ids(text):
    """Wrap emails and IDs with markers to ensure they are treated as single tokens."""
    text = EMAIL_PATTERN.sub(r"EMAIL_\1_EMAIL", text)
    text = ID_PATTERN.sub(r"ID_\1_ID", text)
    return text


def spacing_correction(text):
    """Applies spacing corrections."""
    for pattern, repl in SPACING_PATTERNS:
        text = re.sub(pattern, repl, text)
    return text


def remove_diacritics_and_punct(text):
    """Removes diacritics and extra punctuation."""
    text = re.sub(DIACRITICS_PATTERN, "", text)  # Remove diacritics
    text = re.sub(EXTRA_PUNCT_PATTERN, "", text)  # Remove punctuation
    return text



def preprocess_single_text(text, expand_abbr=True, preserve_email_id=True, do_spacing_corr=True,
                           remove_diacritics=True):
    if expand_abbr:
        text = expand_abbreviations(text)

    if preserve_email_id:
        text = preserve_emails_ids(text)

    text = normalizer.normalize(text)  # Hazm normalization

    if do_spacing_corr:
        text = spacing_correction(text)

    if remove_diacritics:
        text = remove_diacritics_and_punct(text)

    tokens = word_tokenize(text)  # Tokenize using Hazm
    return tokens

In [48]:
text = "دک علوم پزشکی در ق.ظ برگزار شد."
tokens = preprocess_single_text(text)
print("Test 1 - Basic Abbreviations:", tokens)

text = "برای تماس با ما ایمیل info@example.com یا @example_id را استفاده کنید."
tokens = preprocess_single_text(text, expand_abbr=False, preserve_email_id=True)
print("Test 2 - Emails and IDs:", tokens)

text = "این کتاب ها جدیدتر  ترین نسخه ها هستند."
tokens = preprocess_single_text(text, expand_abbr=False, do_spacing_corr=True)
print("Test 3 - Spacing Corrections:", tokens)

text = "این، یک! متن: آزمایشی است؛ و باید... تست شود؟"
tokens = preprocess_single_text(text, expand_abbr=False, remove_diacritics=True)
print("Test 4 - Remove Diacritics and Punctuation:", tokens)

text = "ایمیل: test@mail.com در ساعت ق.ظ ۹:۰۰ برگزار می‌شود."
tokens = preprocess_single_text(text)
print("Test 5 - Full Features:", tokens)

Test 1 - Basic Abbreviations: ['دانشکده', 'علوم', 'پزشکی', 'در', 'قبل', 'از', 'ظهر', 'برگزار', 'شد']
Test 2 - Emails and IDs: ['برای', 'تماس', 'با', 'ما', 'ایمیل', 'EMAILinfoID@exampleID', 'comEMAIL', 'یا', 'ID@exampleidID', 'را', 'استفاده', 'کنید']
Test 3 - Spacing Corrections: ['این', 'کتاب\u200c\u200cها', 'جدیدتر\u200c\u200cترین', 'نسخه\u200c\u200cها', 'هستند']
Test 4 - Remove Diacritics and Punctuation: ['این', 'یک', 'متن', 'آزمایشی', 'است', 'و', 'باید', 'تست', 'شود']
Test 5 - Full Features: ['ایمیل', 'EMAILtestID@mailID', 'comEMAIL', 'در', 'ساعت', 'قبل', 'از', 'ظهر', '۹۰۰', 'برگزار', 'می\u200cشود']


In [49]:
stemmer = Stemmer()

def stem_tokens(tokens):
    """Applies stemming to a list of tokens."""
    return [stemmer.stem(t) for t in tokens]


def compute_top_k_frequent(tokens, k):
    """
    Computes the top-K most frequent tokens along with their frequencies.
    Returns a dictionary with tokens as keys and their counts as values.
    """
    token_counts = Counter(tokens)
    sorted_tokens = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
    return dict(sorted_tokens[:k])


def preprocess_all_docs(docs, top_k=50):
    """
    Preprocessing pipeline:
      1) Normalize and tokenize each document.
      2) Apply stemming to all tokens across all documents.
      3) Gather all tokens across documents.
      4) Compute top-K frequent tokens with their counts.
      5) Remove top-K frequent tokens.
    """
    all_tokens = {}
    combined_tokens = []

    # Step 1: Normalize and tokenize each document
    for doc_id, doc_data in docs.items():
        content = doc_data['content']
        tokens = preprocess_single_text(content)
        tokens = stem_tokens(tokens)  # Step 2: Apply stemming
        all_tokens[doc_id] = tokens
        combined_tokens.extend(tokens)

    # Step 3: Compute top-K frequent tokens
    top_k_tokens_with_counts = compute_top_k_frequent(combined_tokens, top_k)
    top_k_tokens = set(top_k_tokens_with_counts.keys())  # Extract tokens only

    # Step 4: Remove top-K frequent tokens
    for doc_id in all_tokens:
        all_tokens[doc_id] = [t for t in all_tokens[doc_id] if t not in top_k_tokens]

    return all_tokens, top_k_tokens_with_counts


In [51]:
def main():
    # Load docs
    docs, contents, _ = load_docs()  # Ensure load_docs() is defined elsewhere


    subset_docs = dict(list(docs.items())[:5]) 

    docs, contents, urls = load_docs()
    doc_tokens, freq_words = preprocess_all_docs(docs, top_k=20)

    # Print results
    for doc_id, tokens in doc_tokens.items():
        print(f"Doc {doc_id} final tokens:", tokens)
    print("\nTop frequent words removed:", freq_words)


# Execute the script
main()

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



['گزار', 'گروه', 'سیاس', 'خبرگزار', 'فارس', 'نقل', 'پایگاه', 'اطلاع\u200cرسان', 'ریاس', 'جمهور', 'محسن', 'رضا', 'معاون', 'اقتصاد', 'رئیس\u200cجمهور', 'بعدازظهر', 'دوشنبه', 'دیدار', 'فلیکس', 'پلاسنسیا', 'گونسالس', 'وزیر', 'امور', 'خارجه', 'ونزوئلا', 'ب', 'اینکه', 'امروز', 'ز', 'دول', 'مل', 'ونزوئلا', 'ن', 'دهند', 'همانطور', 'عرصه', 'مقاوم', 'توانمند', 'بودند', 'عرصه', 'سازندگ', 'اقتصاد', 'نیز', 'توانمند', 'هستند', 'اظهار', 'دا', 'ایر', 'همانگونه', 'مرحله', 'مقاوم', 'کنار', 'شما', 'ایستاد', 'عرصه', 'سازندگ', 'پیشرف', 'اقتصاد', 'نیز', 'کنار', 'شما', 'می\u200cایستد', 'تاکید', 'اینکه', 'دو', 'آمادگ', 'دارند', 'مرحله', 'سازندگ', 'گام', 'بلند', 'کنار', 'بردارند', 'گف', 'جمهور', 'اسلام', 'ایر', 'علاقمند', 'ارتقا', 'سطح', 'تعامل', 'ونزوئلا', 'زمینه', 'صدور', 'خدم', 'فن', 'مهندس', 'ساخ', 'پالایشگاه', 'مجتمع', 'پتروشیم', 'استخراج', 'نف', 'تولید', 'فولاد', 'سایر', 'عرصه', 'صنعت', 'معدن', 'معاون', 'اقتصاد', 'رئیس\u200cجمهور', 'ب', 'اینکه', 'آمریکایی\u200c', 'هیچ', 'وجه', 'قابل\u200cاعتماد', 'نیستند